# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846888756946                   -0.70    4.8         
  2   -7.852321058508       -2.27       -1.53    1.0   31.5ms
  3   -7.852614200360       -3.53       -2.55    1.5   35.6ms
  4   -7.852645933612       -4.50       -2.88    2.5   41.2ms
  5   -7.852646493430       -6.25       -3.17    1.0   31.9ms
  6   -7.852646680158       -6.73       -4.10    1.0   31.2ms
  7   -7.852646686473       -8.20       -5.16    1.8   36.4ms
  8   -7.852646686716       -9.61       -5.26    2.2    125ms
  9   -7.852646686728      -10.94       -5.68    1.0   30.8ms
 10   -7.852646686730      -11.70       -6.43    1.0   31.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846640834341                   -0.70           4.5         
  2   -7.852551563895       -2.23       -1.62   0.80    2.0    283ms
  3   -7.852639209090       -4.06       -2.72   0.80    1.0   27.6ms
  4   -7.852646519435       -5.14       -3.38   0.80    2.0   35.7ms
  5   -7.852646682526       -6.79       -4.29   0.80    1.5    140ms
  6   -7.852646686560       -8.39       -4.82   0.80    1.8   32.4ms
  7   -7.852646686719       -9.80       -5.61   0.80    1.8   32.8ms
  8   -7.852646686730      -10.99       -6.54   0.80    2.0   35.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.389958e+01     3.803351e+00
 * time: 0.45978617668151855
     1     1.497460e+00     2.136163e+00
 * time: 0.6877310276031494
     2    -1.402315e+00     2.401464e+00
 * time: 0.7200620174407959
     3    -3.579807e+00     2.370690e+00
 * time: 0.7646591663360596
     4    -4.988087e+00     2.096859e+00
 * time: 0.8094151020050049
     5    -6.479897e+00     1.343088e+00
 * time: 0.8541381359100342
     6    -7.141146e+00     5.712613e-01
 * time: 0.8993279933929443
     7    -7.504259e+00     2.179216e-01
 * time: 0.9290690422058105
     8    -7.696416e+00     1.891440e-01
 * time: 0.9605309963226318
     9    -7.770969e+00     1.367252e-01
 * time: 0.99198317527771
    10    -7.817639e+00     6.541635e-02
 * time: 1.0224010944366455
    11    -7.838642e+00     5.885977e-02
 * time: 1.052950143814087
    12    -7.845037e+00     5.204940e-02
 * time: 1.0822670459747314
    13    -7.846189e+00     3.749749e-02
 * time: 1.112893104553

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846802447126                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645902246                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.05s
  3   -7.852646686730      -13.30       -7.25    204ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.19982795257949e-7
|ρ_newton - ρ_scfv| = 8.994066842169838e-8
|ρ_newton - ρ_dm|   = 5.805381281296187e-9
